In [1]:
from tensorflow.python.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM,GRU
 
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
 
from sklearn.preprocessing import MinMaxScaler
 
from datetime import datetime

from binance.client import Client

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

In [2]:
client = Client('', '')

In [3]:
cmc = CoinMarketCapAPI('  ')

In [4]:
info = client.get_all_tickers()

In [5]:
'Good to check indivudual by get_symbol_info if status is trading'

'Good to check indivudual by get_symbol_info if status is trading'

In [6]:
'Filter the Info to get the list of Symbols ending with USDT'

'Filter the Info to get the list of Symbols ending with USDT'

In [7]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith('USDT')]

In [8]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [9]:
target_rank_info = [d['symbol']+'USDT' for d in rank_info if d['rank'] >= 200 and d['rank'] <= 450]

In [10]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [11]:
klines = client.get_historical_klines('SANDBUSD', Client.KLINE_INTERVAL_1HOUR, "300 day ago UTC") 

In [12]:
array = np.array(klines, dtype=np.float64)

In [13]:
focused_kline_array = np.delete(array, [2, 3, 6, 7, 8, 9, 10, 11], 1)

In [14]:
length = len(focused_kline_array)

In [15]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[2] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value        

In [16]:
percentage_value = find_percentage_increase_or_decrease(focused_kline_array)

In [17]:
percentage_value = np.reshape(percentage_value, (length, 1))

In [18]:
focused_kline_array = np.append(focused_kline_array, percentage_value, 1)

In [19]:
positive_array = focused_kline_array[:,4] >= 0

In [20]:
filtered_positive_array = focused_kline_array[:,4][positive_array]

In [21]:
negative_array = focused_kline_array[:,4] <= 0

In [22]:
filtered_negative_array = focused_kline_array[:,4][negative_array]

In [23]:
average_positive_increase = np.mean(filtered_positive_array)

In [24]:
tolerable_average_positive_increase = average_positive_increase * 2.0

In [25]:
average_negative_increase = np.mean(filtered_negative_array)

In [26]:
tolerable_average_negative_increase = average_negative_increase * 2.0

In [27]:
average_volume = np.mean(focused_kline_array[:,3])

In [28]:
tolerable_volume = average_volume * 2

In [29]:
tolerable_volume

856908.4763823284

In [30]:
tolerable_average_negative_increase

-2.1178675929423503

In [31]:
tolerable_average_positive_increase

2.24912830859279

In [32]:
def is_candidate(filtered_klines_info):
    red_and_volume_check_klines = filtered_klines_info[0:2]
    green_and_volume_check_klines = filtered_klines_info[2:3]

    is_candidate = False
    for red_and_volume_check_kline in red_and_volume_check_klines:
        if float(red_and_volume_check_kline[4]) <= 0 and float(red_and_volume_check_kline[4]) <= tolerable_average_negative_increase and float(red_and_volume_check_kline[3]) > average_volume:
            is_candidate = True
        
    if is_candidate and float(green_and_volume_check_klines[0][4]) > 0 and float(green_and_volume_check_klines[0][4]) >= tolerable_average_positive_increase and float(green_and_volume_check_klines[0][4]) > average_volume:
        is_candidate = True
        
        
    return is_candidate    
    

In [33]:
length = int(len(focused_kline_array))

In [34]:
full_test_data_set = []

for x in range(0, length -3) :

    filtered_klines_info = focused_kline_array[x:x+4]
    investment_in_tether = 10000
    if(len(filtered_klines_info)) == 4 and is_candidate(filtered_klines_info):
        key_test_data_set = [] 
        buy_kline_info = filtered_klines_info[2:3]
        sell_kline_info = filtered_klines_info[3:4]
        buy_quote = buy_kline_info[0][2]
        sell_quote = sell_kline_info[0][2]
        volume = investment_in_tether/buy_quote
 
        
        key_test_data_set.append(sell_kline_info[0][0])
        
        key_test_data_set.append(buy_quote)
        
        key_test_data_set.append(sell_quote)
        
        key_test_data_set.append(volume)
        
        key_test_data_set.append(investment_in_tether)
        
        dis_investment_in_tether = volume * sell_quote
        key_test_data_set.append(dis_investment_in_tether)
        key_test_data_set.append(dis_investment_in_tether - investment_in_tether)
        full_test_data_set.append(key_test_data_set)

In [35]:
full_test_data_set = np.array(full_test_data_set)

In [36]:
profit_or_loss = full_test_data_set[:, 6]

In [37]:
overall_profit = np.sum(profit_or_loss)

In [38]:
overall_profit

11436.913647397221